Extracting Review

In [13]:
import requests
import pandas as pd

games = pd.read_csv("GamesList.csv")
games = games.set_index("id") 

In [14]:
def get_reviews(app_id, num_reviews=100):
    reviews_list = []
    url = f'https://store.steampowered.com/appreviews/{app_id}?json=1&num_per_page=100'
    total_reviews_fetched = 0
    start_offset = 0

    while total_reviews_fetched < num_reviews:
        # Update the URL with the start offset to get the next page of reviews
        paginated_url = f"{url}&start_offset={start_offset}"
        response = requests.get(paginated_url)

        if response.status_code == 200:
            data = response.json()
            reviews = data['reviews']

            if not reviews:
                print("No more reviews available for " + str(games['title'][app_id]))
                break

            # Add reviews to the list
            for review in reviews:
                review_text = review['review']
                votes_up = review['votes_up']
                rating = 'Positive' if review['voted_up'] else 'Negative'

                reviews_list.append({
                    'Review': review_text,
                    'Votes_Up': votes_up,
                    'Rating': rating
                })

            total_reviews_fetched += len(reviews)
            start_offset += len(reviews)  # Increment the offset by the number of reviews fetched

        else:
            print(f"Failed to retrieve reviews: {response.status_code}")
            break

    # Convert the reviews list to a DataFrame
    reviews_df = pd.DataFrame(reviews_list[:num_reviews])  # Limit to the requested number of reviews
    return reviews_df

In [61]:
games = pd.read_csv("GamesList.csv")
games = games.set_index("id") 

ReviewsDataFrame = pd.DataFrame({'Game':[],'Review':[],'Votes_Up':[],'Rating':[]})

rev_count = 300

for id in games.index: 
    reviews_df = get_reviews(id, num_reviews=rev_count)
    if len(reviews_df) > 1:
        game_name = []
        for i in range(rev_count): 
            game_name.append(games['title'][id])
        reviews_df.insert(0, "Game", game_name)
        ReviewsDataFrame = pd.concat([ReviewsDataFrame, reviews_df])


ReviewsDataFrame.dropna
ReviewsDataFrame = ReviewsDataFrame.drop(columns='index')

No more reviews available for Bloons TD Battles 2
No more reviews available for Goose Goose Duck
No more reviews available for Crab Game


<bound method DataFrame.dropna of             Game                                             Review  Votes_Up  \
0      Snakeybus  Don't get me wrong, its a nice game with a coo...       1.0   
1      Snakeybus  Amazing concept for a game. Super simple, but ...       0.0   
2      Snakeybus  Really complex game, super fun to. It might be...       0.0   
3      Snakeybus  This is one of the games you dream about but w...       0.0   
4      Snakeybus       Snakeybus. Just look at that title. Genious.       0.0   
..           ...                                                ...       ...   
295  Bloons TD 6  best game is all i can say (well not best best...       0.0   
296  Bloons TD 6  I have 800+ hours extra on epic games. this is...       0.0   
297  Bloons TD 6  One of the best tower defense games in my opin...       0.0   
298  Bloons TD 6  very fun i really like it a lot do reccomend i...       0.0   
299  Bloons TD 6  sniper monkey goes hard. very fun game would r...       0

In [62]:
#ReviewsDataFrame = ReviewsDataFrame.dropna()
len(ReviewsDataFrame)

10200

In [63]:
ReviewsDataFrame.reset_index(inplace = True)
crabdf = (ReviewsDataFrame['Game'] == 'Crab Game')

In [64]:
ReviewsDataFrame['Game'].unique()

array(['Snakeybus', 'Biped', 'Project Winter - Blackout', 'Worms Rumble',
       'Wobbly Life', 'Party Animals', 'PICO PARK', 'Perfect Heist 2',
       'Stumble Guys', 'Bopl Battle', 'The Riflemen',
       'Scribblenauts Unlimited', 'Pit People®', 'Duck Game',
       'ShellShock Live', 'Worms W.M.D', 'Ratz Instagib',
       'Cat Goes Fishing', 'The Mean Greens - Plastic Warfare',
       'Tower Unite', 'Clustertruck', "Garry's Mod", "Who's Your Daddy?!",
       'Human Fall Flat', 'Happy Room', 'Pikuniku',
       'Stick Fight: The Game', 'Worms_Ultimate_Mayhem', 'Overcooked! 2',
       'Project Winter', 'Super Animal Royale', 'Pummel Party',
       'ibb & obb', 'Bloons TD 6'], dtype=object)

In [72]:
ReviewsDataFrame = ReviewsDataFrame.drop(columns='index')
ReviewsDataFrame

,Game,Review,Votes_Up,Rating
0,Snakeybus,"Don't get me wrong, its a nice game with a coo...",1.0,Negative
1,Snakeybus,"Amazing concept for a game. Super simple, but ...",0.0,Positive
2,Snakeybus,"Really complex game, super fun to. It might be...",0.0,Positive
3,Snakeybus,This is one of the games you dream about but w...,0.0,Positive
4,Snakeybus,Snakeybus. Just look at that title. Genious.,0.0,Positive
...,...,...,...,...
10195,Bloons TD 6,best game is all i can say (well not best best...,0.0,Positive
10196,Bloons TD 6,I have 800+ hours extra on epic games. this is...,0.0,Positive
10197,Bloons TD 6,One of the best tower defense games in my opin...,0.0,Positive
10198,Bloons TD 6,very fun i really like it a lot do reccomend i...,0.0,Positive


In [73]:
ReviewsDataFrame.to_csv("Steam_Reviews_Data.csv")